In [7]:
!pip install opencv-python dlib scipy imutils pygame

In [8]:
import cv2
import dlib
import time
import pygame
from scipy.spatial import distance
from imutils import face_utils


In [9]:
import os
os.getcwd()


'/Users/s.hiteshkarvil'

In [10]:
pygame.mixer.init()
pygame.mixer.music.load("/Users/s.hiteshkarvil/Driver_Drowsiness_Detection/alarm.mp3")

In [11]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])  # vertical
    B = distance.euclidean(eye[2], eye[4])  # vertical
    C = distance.euclidean(eye[0], eye[3])  # horizontal
    return (A + B) / (2.0 * C)

In [12]:
# EAR threshold and number of consecutive frames the eyes must be below the threshold
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20

# Frame counter and alarm flag
counter = 0
alarm_on = False

# Load dlib's face detector and facial landmark predictor
print("[INFO] Loading the predictor and detector...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/s.hiteshkarvil/Desktop/Driver_Drowsiness_Detection/shape_predictor_68_face_landmarks.dat")

# Get indexes for the left and right eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]


[INFO] Loading the predictor and detector...


In [14]:
# Start video stream
print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
time.sleep(1.0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        # Draw eye contours
        cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)

        # Drowsiness detection logic
        if ear < EAR_THRESHOLD:
            counter += 1
            if counter >= EAR_CONSEC_FRAMES:
                if not alarm_on:
                    alarm_on = True
                    pygame.mixer.music.play(-1)
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            counter = 0
            if alarm_on:
                alarm_on = False
                pygame.mixer.music.stop()

        # Display EAR
        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)

    # Show frame
    cv2.imshow("Driver Drowsiness Detection", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()


[INFO] Starting video stream...


2025-04-07 16:20:27.352 python[44358:7137818] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-07 16:20:27.353 python[44358:7137818] +[IMKInputSession subclass]: chose IMKInputSession_Modern
